# ECSE 415 - Assignment 4: Object Recognition
## Road Signs Detection

**Dataset:** Road Signs Detection (Kaggle)

**Task:** Implement and evaluate object detection models for traffic sign recognition

## 0. Environment Setup

### 0.1 CUDA Verification

In [ ]:
import torch

cuda_available = torch.cuda.is_available()
device = torch.device('cuda' if cuda_available else 'cpu')

print(f"CUDA Available: {cuda_available}")
if cuda_available:
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Device: {device}")

### 0.2 Install Dependencies

In [ ]:
!pip install -q kaggle ultralytics torchvision torchmetrics pycocotools

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

### 0.3 Configuration

In [ ]:
# Class mapping
CLASS_NAMES = {
    0: 'Speed Limit 80', 1: 'Speed Limit 50', 2: 'Green Light', 3: 'Speed Limit 90',
    4: 'Speed Limit 40', 5: 'Speed Limit 120', 6: 'Stop', 7: 'Speed Limit 60',
    8: 'Speed Limit 70', 9: 'Speed Limit 20', 10: 'Speed Limit 110', 11: 'Red Light',
    12: 'Speed Limit 30', 13: 'Speed Limit 100'
}
NUM_CLASSES = 14

# Paths
DATA_DIR = Path('/content/traffic_signs')
TRAIN_IMG_DIR = DATA_DIR / 'train' / 'images'
TRAIN_LBL_DIR = DATA_DIR / 'train' / 'labels'
TEST_IMG_DIR = DATA_DIR / 'test' / 'images'

# Training parameters
IMG_SIZE = 416
BATCH_SIZE = 16
EPOCHS = 50
LR = 0.001
TRAIN_VAL_SPLIT = 0.7

## 1. Data Preparation

### 1.1 Download Dataset from Kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Setup Kaggle API credentials
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download and extract dataset
# Replace 'competition-name' with actual competition name
# !kaggle competitions download -c road-signs-detection
# !unzip -q road-signs-detection.zip -d /content/traffic_signs

### 1.2 Dataset Exploration

In [ ]:
# Count images and annotations
train_images = list(TRAIN_IMG_DIR.glob('*.jpg'))
train_labels = list(TRAIN_LBL_DIR.glob('*.txt'))
test_images = list(TEST_IMG_DIR.glob('*.jpg'))

print(f"Training images: {len(train_images)}")
print(f"Training labels: {len(train_labels)}")
print(f"Test images: {len(test_images)}")

In [ ]:
# Parse YOLO annotations and count class distribution
def parse_yolo_labels(label_dir):
    class_counts = {i: 0 for i in range(NUM_CLASSES)}
    total_objects = 0
    
    for label_file in Path(label_dir).glob('*.txt'):
        with open(label_file, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])
                class_counts[class_id] += 1
                total_objects += 1
    
    return class_counts, total_objects

class_counts, total_objects = parse_yolo_labels(TRAIN_LBL_DIR)
print(f"Total objects: {total_objects}")

In [ ]:
# Plot class distribution
classes = [CLASS_NAMES[i] for i in range(NUM_CLASSES)]
counts = [class_counts[i] for i in range(NUM_CLASSES)]

plt.figure(figsize=(12, 6))
plt.bar(range(NUM_CLASSES), counts)
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(range(NUM_CLASSES), classes, rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize sample images with bounding boxes
def draw_yolo_boxes(img_path, label_path, class_names):
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:5])
            
            # Convert YOLO format to pixel coordinates
            x1 = int((x_center - width/2) * w)
            y1 = int((y_center - height/2) * h)
            x2 = int((x_center + width/2) * w)
            y2 = int((y_center + height/2) * h)
            
            # Draw box and label
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = class_names[class_id]
            cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return img

# Display 5 sample images
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for i, ax in enumerate(axes):
    img_path = train_images[i]
    label_path = TRAIN_LBL_DIR / f"{img_path.stem}.txt"
    
    if label_path.exists():
        img = draw_yolo_boxes(img_path, label_path, CLASS_NAMES)
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(f"Sample {i+1}")

plt.tight_layout()
plt.show()

### 1.3 Train/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

# Split image paths
train_imgs, val_imgs = train_test_split(
    train_images, 
    train_size=TRAIN_VAL_SPLIT, 
    random_state=42
)

print(f"Training set: {len(train_imgs)} images")
print(f"Validation set: {len(val_imgs)} images")

## 2. Baseline Model - YOLOv8

### 2.1 Prepare Data Configuration

In [ ]:
# Create YOLO data.yaml configuration
data_yaml = f"""
path: {DATA_DIR}
train: train/images
val: train/images

nc: {NUM_CLASSES}
names: {list(CLASS_NAMES.values())}
"""

with open(DATA_DIR / 'data.yaml', 'w') as f:
    f.write(data_yaml)

### 2.2 Train YOLOv8

In [ ]:
from ultralytics import YOLO

# Load pretrained YOLOv8 model
model_yolo = YOLO('yolov8n.pt')

# Train
results = model_yolo.train(
    data=str(DATA_DIR / 'data.yaml'),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=0 if cuda_available else 'cpu',
    project='yolo_baseline',
    name='train'
)

### 2.3 Evaluate YOLOv8

In [ ]:
# Validation metrics
metrics = model_yolo.val()

print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")

In [ ]:
# Visualize predictions on validation set
results = model_yolo.predict(source=str(val_imgs[0]), save=True, conf=0.5)

## 3. Custom Model Implementation

### 3.1 Model Architecture

Implement custom object detection model (e.g., RetinaNet, Faster R-CNN)

In [ ]:
# Custom model implementation placeholder
# TODO: Implement custom detection model

### 3.2 Training

In [ ]:
# Training loop placeholder
# TODO: Implement training

## 4. Model Evaluation

### 4.1 Performance Metrics

In [ ]:
# Evaluation placeholder
# TODO: Compute F1, mAP@50, mAP@50-95, confusion matrix

### 4.2 Visualization

In [ ]:
# Visualization placeholder
# TODO: Display predictions on validation images

### 4.3 Generalization Test

In [ ]:
# External image test placeholder
# TODO: Test on external traffic sign image

## 5. Kaggle Submission

### 5.1 Generate Predictions

In [ ]:
# Generate predictions on test set
predictions = []

for img_path in test_images:
    results = model_yolo.predict(source=str(img_path), conf=0.25, verbose=False)
    
    # Extract boxes
    for idx, result in enumerate(results[0].boxes.data.cpu().numpy()):
        x1, y1, x2, y2, conf, class_id = result
        
        # Convert to YOLO format (normalized)
        w, h = IMG_SIZE, IMG_SIZE
        x_center = ((x1 + x2) / 2) / w
        y_center = ((y1 + y2) / 2) / h
        width = (x2 - x1) / w
        height = (y2 - y1) / h
        
        predictions.append({
            'ID': f"{img_path.stem}_{idx}",
            'class_label': int(class_id),
            'x_center': x_center,
            'y_center': y_center,
            'width': width,
            'height': height
        })

### 5.2 Create Submission File

In [ ]:
# Create submission DataFrame
submission_df = pd.DataFrame(predictions)

# Ensure all required IDs are present (match sample_submission.csv)
# Load sample submission if available
# sample_submission = pd.read_csv('sample_submission.csv')
# submission_df = submission_df.merge(sample_submission[['ID']], on='ID', how='right')

# Save
submission_df.to_csv('submission.csv', index=False)
print(f"Submission file created with {len(submission_df)} predictions")

In [ ]:
# Submit to Kaggle
# !kaggle competitions submit -c road-signs-detection -f submission.csv -m "Submission message"